<h3>Compartment: Name</h3>

Enclosure area

<img src="pic.png" style="width: 500px;"/>

In [4]:
import numpy as np
import pandas as pd
import sympy as sp
import handcalcs.render
from handcalcs import handcalc
import forallpeople as si
import matplotlib.pyplot as plt
%matplotlib inline
from math import tan, sin,cos, pi, radians, sqrt, trunc
from IPython.display import display, Markdown
si.environment('thermal', top_level=True)

In [5]:
def Fire_load(building_type):
    # Fire load densities according to EN 1991-1-2 DK Annex
    if building_type == 'hospital' or building_type == 'cinema' or building_type == 'lecture hall':
        q = 150  # MJ/m^2
    elif building_type == 'office' or building_type == 'residence':
        q = 200  # MJ/m^2
    elif building_type == 'car park':
        q = 300  # MJ/m^2
    else:
        raise ValueError("Please input a valid building type")
    return q*10**6*J_m2

<h4>Fuel load density</h4>

In [6]:
q = Fire_load('lecture hall')

In [7]:
%%render 2
q

<IPython.core.display.Latex object>

<h4>Opening factor<h4>

Enclosure area

In [8]:
%%render param 2
#Area of the "cube" of the compartment

B = 5.5*m
L = 4.5*m
H = 3.5*m


<IPython.core.display.Latex object>

In [9]:
%%render 2

A_f = B*L
A_c = A_f
A_w_1 = H*L
A_w_2 = H*B
A_t = A_f + A_c + 2*(A_w_1 + A_w_2)

<IPython.core.display.Latex object>

Air flow factor

In [15]:
%%render symbolic 2

h_win_i = 2.052*m #height of window, i
w_win_i = 0.8*m #width of window, i

h_door_i = 2.1*m#height of door, i
w_door_i = 0.74*m#width of door, i


<IPython.core.display.Latex object>

In [17]:
%%render symbolic 2

A_win_i = h_win_i*w_win_i #area of window,i
A_door_i = h_door_i*w_door_i #area of door,i
A_o = A_win_i + A_door_i #Total opening area
h_av = (h_win_i*A_win_i+h_door_i*A_door_i)/A_o #Average opening height
AF = (A_o*sqrt(h_av))*m**(1/2) #Air flow factor
O = max((AF/A_t).prefix('unity'),(0.02*m**0.5).prefix('unity')) #Opening factor

<IPython.core.display.Latex object>

In [ ]:
(5.52*2.1+5.52*2.1)/11.04

In [ ]:
%%render 2
t_max = 0.0078*(q.value/10**6)/O.value

<h4>Thermal inertia</h4>

In [ ]:
%%render 2

rho = 2300*kg/(m**3)
c = 1000*J/(kg*K)
lamb = 0.8*W/(m*K)
b = (rho*c*lamb)**0.5

* For different layers in the wall

In [ ]:
def diff_layers(s1,rho_l_1,c_l_1,lamb_l_1,rho_l_2,c_l_2,lamb_l_2):
    """
    Layer on the left is fire exposed and layer number 1
    s1 = thickness of layer 1 in meters
    rho_l_1 = density of layer 1 in kg/m^3
    c_l_1 = specific heat of layer 1 in J/kg/K
    lamb_l_1 = thermal conductivity of layer 1 in W/m/K
    """
    b_l_1 = (rho_l_1*c_l_1*lamb_l_1)**0.5
    b_l_2 = (rho_l_2*c_l_2*lamb_l_2)**0.5
    if b_l_1 < b_l_2:
        b_w_1 = b_l_1
    else:
        """
        Penetration depth of the enclosure
        """
        D = lamb_l_1/(c_l_1*rho_l_1) #Thermal diffusivity
        s_lim = (D*t_max*60)**0.5 #penetration depth
        if s1 <= s_lim:
            b_w_1 = s1*b_l_1/s_lim + b_l_2*(1-s1/s_lim)
        else:
            b_w_1 = b_l_1
    return b_w_1
        

* For different walls in the enclosure

In [ ]:
%%render 2
b_f = 1
b_w = 1
b_c = 1
b_t = (A_f*b_f + A_w_1*b_w + A_w_2*b_w + A_c*b_c)/A_t

Parametric curve validity check

In [ ]:
if B*L >= 500: 
    display(Markdown('Compartment size not compatible with model.'))
elif H >= 4:
    display(Markdown('Compartment height not compatible with model.'))
elif not(0.019 < O <= 0.2):
    display(Markdown('Opening factor not compatible with model.'))
elif not(100<=b_t<=2200):
    display(Markdown('Thermal inertia not compatible with model.'))
elif not(50*10**9 < q*(A_t) < 1000*10**9):
    display(Markdown('Fuel load not compatible with model.'))
else:
    display(Markdown('Compartment size, height, opening factor, thermal inertia, and fuel load are compatible with model.'))

In [ ]:
def gamma_f(O,b):
    return (O.value/0.04/(b.value/1160))**2

In [ ]:
gamma = gamma_f(O,b)

In [ ]:
t_max = 0.0078*(q.value/10**6)/O.value
t = np.arange(0,t_max*5)
T = 20+345*np.log10(8*gamma_f(O,b)*t+1)/(1+0.04*(t/t_max)**3.5)
T_max = T.max()
plt.style.use('ggplot')
plt.plot(t,T, label="DS parametric fire curve", color = 'red')
plt.ylim(ymax=max(T)*1.2)
plt.legend()
plt.xlabel("Time [min]")
plt.ylabel("Temperature [C]")
plt.text(0.5 * (0 + t_max), 0.1 * (0 + T_max),
     r"$T=20 + \frac{345\cdot log_{10}(8\Gamma \cdot t+1)}{1+0.04(t/t_{max})^{3.5}}$", horizontalalignment='left',
     fontsize=20)
plt.show()